# Workbook for automated creation  of CMWP files for a pre-integrated Zr XRD pattern

This workbook takes a folder of .dat files and creates a .bg-spline.dat and .pkea-index.dat file based on Zr peak positions.

## Import stuff

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt
import glob
import csv
from scipy.interpolate import CubicSpline
from shutil import copyfile
from src.xrd_tools import getReflections

## Define settings

In [ ]:
base = '/home/rhys/Documents/CMWP-200623/testdata/'         # This is the base directory where the data is stored
template = base + '0template/'                              # This is where the template files are stored. These are copied for each .dat file.

wavelength = 1.5406                                         # Wavelength in angstrom
a=3.232                # a lattice paramater in angstrom
c=5.147                # c lattice paramater in angstrom

baseline=[42, 52, 61, 72, 85, 101.5]                        # What 2theta points to calculte the background spline from

searchrange = 20                                            # How many data points each side of the approximate 2theta peak position to search for the true peak


## Find files

In [ ]:
files = glob.glob(base + '*.dat')
template_files = glob.glob(template + '*')

files = [x for x in files if 'bg-spline' not in x]
files = [x for x in files if 'peak-index' not in x]
    
print('Found {0} files'.format(len(files)))
for file in files: print('\t{0}'.format(file))
print('\nFound {0} template files:'.format(len(template_files)))
for file in template_files: print('\t{0}'.format(file))

## Run

In [ ]:
peak_name, peak_pos = getReflections(crystalType='hcp', a=a, c=c, wavelength=wavelength, printReflections=False)

for file in files:
    
    for t_file in template_files:
         copyfile(t_file, file.split('.')[-2] + t_file.split('/')[-1][8:])
        
    ########################### Load data ###########################
    
    data = np.loadtxt(file)
    xvals = data[:,0]
    yvals = data[:,1]

    ########################### Make figure ###########################

    plt.ioff()
    fig, (ax) = plt.subplots(1, 1, figsize=(16,8))
    ax.set_title(file.split('.')[-2].split('/')[-1]);
    ax.set_xlabel('2theta (deg)', fontsize=14); 
    ax.set_ylabel('Intensity', fontsize=14);
    plt.subplots_adjust(left=0.1, bottom=0.1, top=0.9, right=0.9, wspace=0.115, hspace=0)
    plt.subplots_adjust(hspace=0)
    
    ax.plot(xvals, yvals)
    ax.set_xlim(np.min(xvals)+0.01, np.max(xvals)-0.01)
    ax.set_ylim(np.min(yvals)-5, np.max(yvals)*1.1)

    ########################### Make bg-spline ###########################
    
    baseline_int = []
    
    baseline_new = baseline + [np.min(xvals)+(xvals[1]-xvals[0])*8, np.max(xvals)-(xvals[1]-xvals[0])*8]

    for j in baseline_new:
        num_index=np.argmin(np.abs(xvals-j))
        baseline_int.append(np.mean(yvals[num_index-5:num_index+5])*0.99)

    baseline_new, baseline_int = (list(t) for t in zip(*sorted(zip(baseline_new, baseline_int))))

    with open(file.split('.')[-2]+'.bg-spline.dat', 'w') as f:
        np.savetxt(fname = f, X=np.transpose([baseline_new, baseline_int]), fmt = ('%1.5f'))

    xs = np.arange(start=np.min(xvals), stop=np.max(xvals), step=0.1)
    cs = CubicSpline(baseline_new, baseline_int)

    ax.plot(xs, cs(xs))
    ax.plot(baseline_new, baseline_int, 'o',c='r')
        
    ########################### Make peak-index ###########################

    with open(file.split('.')[-2]+'.peak-index.dat', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        for name, pos in zip(peak_name, peak_pos):

            if np.min(xvals) < pos < np.max(xvals):
                
                # approx peak postion
                approx_peak_index = np.argmin(np.abs(xvals-pos))

                # get actual peak position
                peak_index = np.argmax(yvals[approx_peak_index-searchrange:approx_peak_index+searchrange])+approx_peak_index-searchrange
                yval = yvals[peak_index]

                # draw line and print name
                ax.axvline(xvals[peak_index], alpha=0.1, c='r')
                ax.text(xvals[peak_index], yval+10, name, horizontalalignment = 'center', c='r')

                intensity = yval - cs(xvals)[peak_index]

                writer.writerow(['{0:.4f} {1:.1f} {2} 0'.format(xvals[peak_index], intensity, name)])

    plt.yscale('log')
    plt.savefig(file.split('.')[-2]+'_log.png', dpi=200)
    plt.close()